In [ ]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import plotly.offline as py
import json
import plotly.express as px
import webbrowser

from dash import Dash, dcc, html, Input, Output

# Import library with current code functions
sys.path.append(os.path.join("..", "lib"))
import manual_labeler_functions as man_lab_fun, general_functions as gf, files_paths as fp

## Function to analyzer the graphs of the selected video series

In [ ]:
def PLOT_MEASURE_EXPLORER(df):

    port = 5000
    external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
    app = Dash(__name__, external_stylesheets=external_stylesheets)

    def open_browser():
    	webbrowser.open_new("http://localhost:{}".format(port))
          
    app.layout = html.Div([
        html.H4('Measure Explorer'),
        html.H6('Reference Signal'),
        html.I('Setting the frame interval range'),
        html.Br(),
        dcc.Input(id='input-init', type='number', min=df.index.min(), max=df.index.max(), value=df.index.min()),
        dcc.Input(id='input-end', type='number', value=df.index.max()),
        dcc.Graph(id="graph"),
        dcc.Checklist(
            id="checklist",
            options=df.columns,
            value=["m1", "m3"],
            inline=True
        ),
        dcc.Store(
        id='data-output',
        data=[{
            'measures': ["m1", "m3"],
            'init_label': df.index.min(),
            'end_label': df.index.max()
        }]
        ),
        html.Br(),
        html.Details([
        html.Summary('Input Parameters'),
        dcc.Markdown(id='clientside-figure-json')
        ])
    ])
    
    @app.callback(
        Output("graph", "figure"), 
        Input("checklist", "value"),
        Input("input-init", "value"),
        Input("input-end", "value"))
    def update_line_chart(measures, frame_init, frame_end):
        mask = df.columns.isin(measures)
        filtered_df = df.loc[frame_init:frame_end, mask]
        fig = px.line(filtered_df, width=900, height=400, labels={
                     "frame_seq": "<b>Frame number</b>",
                     "value": "<b>Amplitude (pixels)</b>",
                     "variable": "<b>Measures</b>"
                 },)
        data = measures
        fig.update_layout(
            plot_bgcolor='white',
            font_size=12,
        )
        fig.update_xaxes(
            mirror=True,
            ticks='outside',
            showline=True,
            linecolor='black',
            gridcolor='lightgrey',
        )
        fig.update_yaxes(
            mirror=True,
            ticks='outside',
            showline=True,
            linecolor='black',
            gridcolor='lightgrey',
        )
        return fig

    @app.callback(
        Output('data-output', 'data'),
        Input("checklist", "value"),
        Input("input-init", "value"),
        Input("input-end", "value"))
    def update_store_data(measures, frame_init, frame_end):
        mask = df.columns.isin(measures)
        filtered_df = df.loc[:,mask]
        return [{
            'measures': filtered_df.columns,
            'init_label': frame_init,
            'end_label': frame_end
        }]
    
    
    @app.callback(
        Output('clientside-figure-json', 'children'),
        Input('data-output', 'data')
    )
    def generated_data_json(data):
        return '```\n'+json.dumps(data, indent=2)+'\n```'
   
    if __name__ == '__main__':
        app.run_server(debug=True, port=port)

## Getting all the CSV files in the REF-Gold-Label

In [ ]:
FILE_LIST_VD_MEASURE = gf.find_files_in_all_subdirectories([fp.DATASET_SEED], fp.VD_INFO)

In [ ]:
FILE_LIST_VD_MEASURE

## Generates an index with information about reference series.

In [ ]:
INDEX_DT = man_lab_fun.CREATE_LABELED_INDEX(FILE_LIST_VD_MEASURE)

In [ ]:
INDEX_DT

## Manual Process

### Selects the video for analysis and CSV data file

In [ ]:
# Type the video id to label
VIDEO_ID = 3

VIDEO_FILE_PATH = str(INDEX_DT.loc[VIDEO_ID,'path'])

# Read CSV data file
VD_MEASURE_DT = man_lab_fun.READ_CSV_FILE(VIDEO_FILE_PATH, fp.VD_MEASURE_L0)

# set frames_seq as index
VD_MEASURE_DT_V2 = VD_MEASURE_DT.set_index(pd.Index(VD_MEASURE_DT['frame_seq']))
VD_MEASURE_DT_V2.drop(columns=["frame_seq", "video_id", "time_seconds"], inplace=True)

In [ ]:
VD_MEASURE_DT

### Graph Analyse

In [ ]:
PLOT_MEASURE_EXPLORER(VD_MEASURE_DT_V2)

### Loads frames from the selected video for further image analysis 


In [ ]:
VIDEO_EXT = '.mp4'

# Get path from selected video id
video_name = str(INDEX_DT.loc[VIDEO_ID,'link_video']) + VIDEO_EXT

path_video_file = os.path.join(fp.VIDEO_SOURCE_SEED, video_name)

video_frames = man_lab_fun.LOAD_VIDEO_FRAMES(path_video_file)

### Set Frames Parameters

In [ ]:
start_frame_crop = 0
end_frame_crop = len(VD_MEASURE_DT) - 1
n_frames_per_row = 6

### Displays frames for image analysis

In [ ]:
man_lab_fun.DISPLAY_FRAMES(video_frames, start_frame_crop, end_frame_crop, n_frames_per_row)

### Reads or creates the VD_LABELED file if it does not exist.

In [ ]:
labeled_file_exists = int(str(INDEX_DT.loc[VIDEO_ID,'label_file_exist']))

if labeled_file_exists == 0:
    print('Creating Label File...')
    VD_MEASURE_DT['label_measures'] = str({})
    VD_LABELED_DT = VD_MEASURE_DT.copy()
else:
    print('Reading Label File...') 
    VD_LABELED_DT = man_lab_fun.READ_CSV_FILE(str(INDEX_DT.loc[VIDEO_ID,'path']).split()[2], fp.VD_LABELED_L0)

### Label the selected frames

In [ ]:
# Type the Begin and End frames you want to label (By default label the section with the selected frames above)
init_frame_label = start_frame_crop
end_frame_label = end_frame_crop

# Prepare List and Dict
selected_class = 'neutral'
label_measure_insert = ["m1", "m3", "m12", "e1", "e2"]

# Just Insert
VD_LABELED_DT = man_lab_fun.UPDATE_LABEL_DF(init_frame_label, end_frame_label, selected_class, label_measure_insert, VD_LABELED_DT)

### Verify the added labels.

In [ ]:
try:
    get_measure = man_lab_fun.GET_MEASURES_FROM_CLASS (VD_LABELED_DT, selected_class)
    print(get_measure)
except:
    print('Class not found')

### Plots a graph marking the start and end of the labels for a class.

In [ ]:
# Select the begin and end frame to be shown
start_frame = 0
end_frame = len(VD_LABELED_DT)

py.init_notebook_mode(connected=True)
plt.rcParams['font.size'] = 10
plt.rcParams["font.family"] = "Times New Roman"

man_lab_fun.PLOT_CLASS_GRAPH(VD_LABELED_DT, VD_MEASURE_DT_V2, selected_class, start_frame, end_frame)

### Save the VD_LABELED file to disk

In [ ]:
VD_LABELED_DT.to_csv(os.path.join(VIDEO_FILE_PATH, fp.VD_LABELED_L0))